In [1]:
import requests as req
import re
import json 
import datetime as dt
from dateutil.relativedelta import relativedelta
import pandas as pd
import numpy as np
import time
import secedgar
from secedgar.cik_lookup import get_cik_map
key = json.load(open('keys.json'))['key']

In [2]:


# Create date range to query
dateRange = {
    'rangeStartDate' : dt.datetime.strptime('2020-01-01','%Y-%m-%d'),
    'rangeEndDate' : dt.datetime.strptime('2021-04-01','%Y-%m-%d'),
} 
print(dateRange['rangeStartDate'],dateRange['rangeEndDate'])

2020-01-01 00:00:00 2021-04-01 00:00:00


# SEC API
https://www.sec.gov/edgar/sec-api-documentation
http://xbrl.squarespace.com/understanding-sec-xbrl-financi/

In [3]:
tickers = [
'BAC',
'WFC',
'GS',
'USB',
'COF',
'HBAN',
'MTB',
'FICO',
'FNMA'
]

tickersdf = pd.DataFrame(tickers, columns=['ticker'])
ciks = pd.DataFrame((list(get_cik_map()["ticker"].items())), columns=['ticker','CIK'])
# ciks
tickers_CIKs = tickersdf.merge(ciks, left_on='ticker', right_on='ticker')


In [4]:
# Dataframe setup
#Columns: Ticker | CIK | Date | Revenue | Net Income
# columns = ['ticker', 'CIK', 'filing_date','revenue', 'net_income']
tickersData = pd.DataFrame(tickers_CIKs, columns=['ticker', 'CIK'])
#add leading zeros
tickersData['CIK'] = tickersData['CIK'].apply(lambda x: x.zfill(10))
tickersData


,ticker,CIK
0,BAC,0000070858
1,WFC,0000072971
2,GS,0000886982
3,USB,0000036104
4,COF,0000927628
5,HBAN,0000049196
6,MTB,0000036270
7,FICO,0000814547
8,FNMA,0000310522


In [5]:
# queryString = (f'https://data.sec.gov/api/xbrl/companyfacts/CIK0000070858.json')
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
#     'From': 'youremail@domain.com'  # This is another valid field
# }
# data = re.get(queryString, headers = headers)
# data
# results = data.json()
# results

In [6]:
def get_results(ticker, CIK, start_date, end_date, xbrl_item):
    queryString = (f'https://data.sec.gov/api/xbrl/companyconcept/CIK{str(CIK)}/us-gaap/{xbrl_item}.json')
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
        'From': 'eth22mer@gmail.com'
    }
    data = req.get(queryString, headers = headers)
    if str(data) == '<Response [404]>':
        print(str(data))
        return pd.DataFrame(data = [{'ticker':ticker, 'CIK':CIK, 'end':np.nan, xbrl_item:np.nan}])
    # print(data.json())
    results = data.json().get('units').get('USD')
    results = pd.DataFrame(results)
    results = results[results.end<=end_date]
    results = results[results.start>=start_date]
    # results = results[results.form == "10-Q"]
    # results = results[results.frame.notnull()]
    results['CIK'] = CIK
    results['ticker'] = ticker
    results = results.rename(columns = {'val':xbrl_item})
    # print(results[['ticker', 'CIK', 'end', xbla_item]])
    # return results[['ticker', 'CIK', 'end', 'frame', xbla_item]]
    return results


In [7]:
all_results = pd.DataFrame([])
for row in tickersData.iterrows():
    print(row[1]['ticker'])
    all_results = all_results.append(get_results(row[1]['ticker'], row[1]['CIK'], '2020-01-01', '2022-09-01', 'NetIncomeLoss'))
    time.sleep(1)
all_results

BAC
WFC
GS
USB
COF
HBAN
MTB
FICO
FNMA


,start,end,NetIncomeLoss,accn,fy,fp,form,filed,frame,CIK,ticker
162,2020-01-01,2020-03-31,4010000000,0000070858-20-000023,2020,Q1,10-Q,2020-05-01,NaN,0000070858,BAC
163,2020-01-01,2020-03-31,4010000000,0000070858-21-000063,2021,Q1,10-Q,2021-04-29,CY2020Q1,0000070858,BAC
164,2020-01-01,2020-06-30,7543000000,0000070858-20-000040,2020,Q2,10-Q,2020-07-31,NaN,0000070858,BAC
165,2020-01-01,2020-06-30,7543000000,0000070858-21-000084,2021,Q2,10-Q,2021-07-30,NaN,0000070858,BAC
166,2020-04-01,2020-06-30,3533000000,0000070858-20-000040,2020,Q2,10-Q,2020-07-31,NaN,0000070858,BAC
...,...,...,...,...,...,...,...,...,...,...,...
211,2021-01-01,2021-06-30,12145000000,0000310522-21-000387,2021,Q2,10-Q,2021-08-03,NaN,0000310522,FNMA
212,2021-04-01,2021-06-30,7152000000,0000310522-21-000387,2021,Q2,10-Q,2021-08-03,CY2021Q2,0000310522,FNMA
213,2021-01-01,2021-09-30,16987000000,0000310522-21-000557,2021,Q3,10-Q,2021-10-29,NaN,0000310522,FNMA
214,2021-07-01,2021-09-30,4842000000,0000310522-21-000557,2021,Q3,10-Q,2021-10-29,CY2021Q3,0000310522,FNMA


In [8]:
## add a column that has years correctly
all_results['frame_fy'] = all_results['frame'].str.extract(r'(\d{4})')
all_results

,start,end,NetIncomeLoss,accn,fy,fp,form,filed,frame,CIK,ticker,frame_fy
162,2020-01-01,2020-03-31,4010000000,0000070858-20-000023,2020,Q1,10-Q,2020-05-01,NaN,0000070858,BAC,NaN
163,2020-01-01,2020-03-31,4010000000,0000070858-21-000063,2021,Q1,10-Q,2021-04-29,CY2020Q1,0000070858,BAC,2020
164,2020-01-01,2020-06-30,7543000000,0000070858-20-000040,2020,Q2,10-Q,2020-07-31,NaN,0000070858,BAC,NaN
165,2020-01-01,2020-06-30,7543000000,0000070858-21-000084,2021,Q2,10-Q,2021-07-30,NaN,0000070858,BAC,NaN
166,2020-04-01,2020-06-30,3533000000,0000070858-20-000040,2020,Q2,10-Q,2020-07-31,NaN,0000070858,BAC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
211,2021-01-01,2021-06-30,12145000000,0000310522-21-000387,2021,Q2,10-Q,2021-08-03,NaN,0000310522,FNMA,NaN
212,2021-04-01,2021-06-30,7152000000,0000310522-21-000387,2021,Q2,10-Q,2021-08-03,CY2021Q2,0000310522,FNMA,2021
213,2021-01-01,2021-09-30,16987000000,0000310522-21-000557,2021,Q3,10-Q,2021-10-29,NaN,0000310522,FNMA,NaN
214,2021-07-01,2021-09-30,4842000000,0000310522-21-000557,2021,Q3,10-Q,2021-10-29,CY2021Q3,0000310522,FNMA,2021


In [33]:
arg = all_results.copy()
# arg = arg.drop_duplicates()
arg['NetIncomeLoss'] = all_results.apply(lambda x: x['NetIncomeLoss']*-1 if x['form'] == '10-Q'  else x['NetIncomeLoss'], axis=1)
# arg['fpCLEAN'] = arg.apply(lambda x: np.nan if x['frame'] == '10-K'  else x['frame'][6:8], axis=1)
# arg['fyCLEAN']  = arg.apply(lambda x: x['frame'][2:6], axis=1)
# arg.sort_values(['ticker', 'fyCLEAN', 'fpCLEAN'], ascending=[True, True, True])
arg = arg.groupby(by='ticker').apply(lambda grp: grp.groupby(by='frame_fy')[['NetIncomeLoss']].sum())

# tickersData['CIK'].apply(lambda x: x.zfill(10))
# df['A'] = df.apply(lambda x: x['B'] if x['A']==0 else x['A'], axis=1)
all_results
# arg = arg[['ticker', 'frame', 'NetIncomeLoss']]
# arg.to_csv(f'all_results_qtrs4.csv')

,start,end,NetIncomeLoss,accn,fy,fp,form,filed,frame,CIK,ticker,frame_fy
162,2020-01-01,2020-03-31,4010000000,0000070858-20-000023,2020,Q1,10-Q,2020-05-01,NaN,0000070858,BAC,NaN
163,2020-01-01,2020-03-31,4010000000,0000070858-21-000063,2021,Q1,10-Q,2021-04-29,CY2020Q1,0000070858,BAC,2020
164,2020-01-01,2020-06-30,7543000000,0000070858-20-000040,2020,Q2,10-Q,2020-07-31,NaN,0000070858,BAC,NaN
165,2020-01-01,2020-06-30,7543000000,0000070858-21-000084,2021,Q2,10-Q,2021-07-30,NaN,0000070858,BAC,NaN
166,2020-04-01,2020-06-30,3533000000,0000070858-20-000040,2020,Q2,10-Q,2020-07-31,NaN,0000070858,BAC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
211,2021-01-01,2021-06-30,12145000000,0000310522-21-000387,2021,Q2,10-Q,2021-08-03,NaN,0000310522,FNMA,NaN
212,2021-04-01,2021-06-30,7152000000,0000310522-21-000387,2021,Q2,10-Q,2021-08-03,CY2021Q2,0000310522,FNMA,2021
213,2021-01-01,2021-09-30,16987000000,0000310522-21-000557,2021,Q3,10-Q,2021-10-29,NaN,0000310522,FNMA,NaN
214,2021-07-01,2021-09-30,4842000000,0000310522-21-000557,2021,Q3,10-Q,2021-10-29,CY2021Q3,0000310522,FNMA,2021


In [44]:
# pd.DataFrame.from_dict(arg, orient='columns')
# arg.loc[arg['ticker'] == 'BAC']
# arg = arg.reset_index(level=[0,1])
# arg.reset_index(level=[0,1])
arg.loc[arg['ticker'] == 'BAC']



,ticker,frame_fy,NetIncomeLoss
0,BAC,2020,5470000000
1,BAC,2021,7013000000


In [27]:
all_results.loc[(all_results["fp"] == "FY") & (all_results["frame_fy"]=="2021")]

,start,end,NetIncomeLoss,accn,fy,fp,form,filed,frame,CIK,ticker,frame_fy
179,2021-01-01,2021-12-31,31978000000,0000070858-22-000062,2021,FY,10-K,2022-02-22,CY2021,0000070858,BAC,2021
169,2021-01-01,2021-12-31,21548000000,0000072971-22-000096,2021,FY,10-K,2022-02-22,CY2021,0000072971,WFC,2021
172,2021-01-01,2021-12-31,21635000000,0001193125-22-052682,2021,FY,10-K,2022-02-25,CY2021,0000886982,GS,2021
166,2021-01-01,2021-12-31,7963000000,0001193125-22-048709,2021,FY,10-K,2022-02-22,CY2021,0000036104,USB,2021
190,2021-01-01,2021-12-31,12390000000,0000927628-22-000106,2021,FY,10-K,2022-02-25,CY2021,0000927628,COF,2021
241,2021-01-01,2021-12-31,1295000000,0000049196-22-000023,2021,FY,10-K,2022-02-18,CY2021,0000049196,HBAN,2021
170,2021-01-01,2021-12-31,1858746000,0001564590-22-005400,2021,FY,10-K,2022-02-16,CY2021,0000036270,MTB,2021
232,2020-10-01,2021-09-30,392084000,0000814547-21-000019,2021,FY,10-K,2021-11-10,CY2021,0000814547,FICO,2021
215,2021-01-01,2021-12-31,22176000000,0000310522-22-000174,2021,FY,10-K,2022-02-15,CY2021,0000310522,FNMA,2021


In [77]:
# Function that takes a 10k row and what q4 10-q value is and returns a row with that value substituted for the 10-k value
def get_q4_value(row, q4_value):
    print(row, q4_value)

# function that identifies the 10k value for one company for one year, then modifies the vlaue
def get_10k_value(df, dfq4vals, year):
    df10k = df.loc[(df["fp"] == "FY") & (df["frame_fy"]==str(year))]
    dfq4vals = dfq4vals.loc[(dfq4vals["frame_fy"]==str(year))]
    # loop through the dataset with q4 values and find the corresponding row in the 10k set. Then feed this to the get q4_value function
    for ticker in dfq4vals.ticker:
        print(dfq4vals.loc[dfq4vals['ticker'] == ticker])    
        df10k = ((df10k.loc[df10k['ticker'] == ticker, 'NetIncomeLoss']) = (dfq4vals.loc[dfq4vals['ticker'] == ticker, 'NetIncomeLoss']))
        # print(dfq4vals.loc[dfq4vals['ticker'] == ticker, 'NetIncomeLoss'])
    return df10k

get_10k_value(all_results, arg, 2021)


SyntaxError: invalid syntax (585039449.py, line 12)

In [71]:
all_results

,start,end,NetIncomeLoss,accn,fy,fp,form,filed,frame,CIK,ticker,frame_fy
162,2020-01-01,2020-03-31,4010000000,0000070858-20-000023,2020,Q1,10-Q,2020-05-01,NaN,0000070858,BAC,NaN
163,2020-01-01,2020-03-31,4010000000,0000070858-21-000063,2021,Q1,10-Q,2021-04-29,CY2020Q1,0000070858,BAC,2020
164,2020-01-01,2020-06-30,7543000000,0000070858-20-000040,2020,Q2,10-Q,2020-07-31,NaN,0000070858,BAC,NaN
165,2020-01-01,2020-06-30,7543000000,0000070858-21-000084,2021,Q2,10-Q,2021-07-30,NaN,0000070858,BAC,NaN
166,2020-04-01,2020-06-30,3533000000,0000070858-20-000040,2020,Q2,10-Q,2020-07-31,NaN,0000070858,BAC,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
211,2021-01-01,2021-06-30,12145000000,0000310522-21-000387,2021,Q2,10-Q,2021-08-03,NaN,0000310522,FNMA,NaN
212,2021-04-01,2021-06-30,7152000000,0000310522-21-000387,2021,Q2,10-Q,2021-08-03,CY2021Q2,0000310522,FNMA,2021
213,2021-01-01,2021-09-30,16987000000,0000310522-21-000557,2021,Q3,10-Q,2021-10-29,NaN,0000310522,FNMA,NaN
214,2021-07-01,2021-09-30,4842000000,0000310522-21-000557,2021,Q3,10-Q,2021-10-29,CY2021Q3,0000310522,FNMA,2021
